In [1]:
import os

In [2]:
%pwd

'd:\\end_end_mlprojects\\winequal_pred_end2end\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\end_end_mlprojects\\winequal_pred_end2end'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from winequality.constants import *
from winequality.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH):


        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])
    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from winequality.logging import logger
from winequality.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers =  request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )    

            logger.info(f"{filename} download! Wth following info: \n{headers}")

        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")    


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[ 2025-04-26 14:56:27,684  wine_pred_logger - INFO - yaml file: config\config.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/MEGHANASEEPANA/datasets/raw/refs/heads/main/winequality-red.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[ 2025-04-26 14:56:27,690  wine_pred_logger - INFO - yaml file: params.yaml loaded successfully]
[ 2025-04-26 14:56:27,700  wine_pred_logger - INFO - yaml file: schema.yaml loaded successfully]
[ 2025-04-26 14:56:27,706  wine_pred_logger - INFO - created directory at: artifacts]
[ 2025-04-26 14:56:27,710  wine_pred_logger - INFO - created directory at: artifacts/data_ingestion]
[ 2025-04-26 14:56:27,714  wine_pred_logger - INFO - file already exists of size: ~ 25 KB]
